In [1]:
import numpy as np
import cv2

def preprocess_frame(frame):
    # Resize frame to model input size
    resized_frame = cv2.resize(frame, (800, 600))

    # Normalize pixel values
    normalized_frame = resized_frame / 255.0

    # Convert to RGB format
    rgb_frame = cv2.normalize(normalized_frame, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)
    rgb_frame = cv2.cvtColor(rgb_frame, cv2.COLOR_BGR2RGB)

    return rgb_frame

def draw_optical_flow_hsv(flow):
    # Calculate magnitude and angle of flow
    mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])

    # Create HSV image, set saturation
    hsv = np.zeros_like(frame)
    hsv[..., 1] = 255

    # Set hue according to the flow direction
    hsv[..., 0] = ang * 180 / np.pi / 2

    # Set value according to the flow magnitude (normalized)
    hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)

    # Convert HSV to RGB (BGR) format
    bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

    return bgr

# Visualize optical flow
def draw_optical_flow(frame, flow):
    h, w = frame.shape[:2]
    step = 16
    y, x = np.mgrid[step/2:h:step, step/2:w:step].reshape(2,-1).astype(int)
    fx, fy = flow[y,x].T

    # Create image and draw
    vis = cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)
    for (x, y), (fx, fy) in zip(np.stack([x, y], axis=-1), np.stack([fx, fy], axis=-1)):
        cv2.line(vis, (x, y), (x+int(fx), y+int(fy)), (0, 255, 0), 1)
        cv2.circle(vis, (x, y), 1, (0, 255, 0), -1)
    return vis

# Function to stack frames for side-by-side comparison
def stack_frames(original_frame, processed_frame):
    # Resize processed frame to match the original frame size
    processed_resized = cv2.resize(processed_frame, (original_frame.shape[1], original_frame.shape[0]))
    # Stack frames horizontally
    stacked_frame = np.hstack((original_frame, processed_resized))
    return stacked_frame

# Load video
cap = cv2.VideoCapture("video.mp4")

# Read first frame
ret, frame = cap.read()

# Check if video is valid
if not ret:
    print("Error: Could not open video file")
    exit()

# Convert first frame to grayscale
prev_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output2.avi', fourcc, 20.0, (frame.shape[1] * 2, frame.shape[0]))

i = 0
while True:
    # Read next frame
    ret, frame = cap.read()
    if not ret:
        break

    if (i % 10 == 0):
        print(i, "frames completed")
    i += 1

    # Pre-process frame
    frame = preprocess_frame(frame)

    # Convert frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    prev_gray = cv2.resize(prev_gray, (gray.shape[1], gray.shape[0]))
    # Calculate optical flow
    flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    # Visualize optical flow in HSV
    flow_hsv = draw_optical_flow_hsv(flow)

    # Stack original frame and flow visualization
    stacked = stack_frames(frame, flow_hsv)

    # Write the stacked frame to the video file
    out.write(stacked)

    # Update previous frame
    prev_gray = gray

    # Handle keyboard input
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break

# Release everything when job is finished
cap.release()
out.release()
cv2.destroyAllWindows()

0 frames completed
10 frames completed
20 frames completed
30 frames completed
40 frames completed
50 frames completed
60 frames completed
70 frames completed
80 frames completed
90 frames completed
100 frames completed
110 frames completed
120 frames completed
130 frames completed
140 frames completed
150 frames completed
160 frames completed
170 frames completed
180 frames completed
190 frames completed
200 frames completed
210 frames completed
220 frames completed
230 frames completed
240 frames completed
250 frames completed
